In [ ]:
!pip install pyspark
!pip install findspark

^C


In [ ]:
import findspark
findspark.init()


from pyspark.sql import SparkSession
from pyspark.sql import Row

In [ ]:
spark = SparkSession.builder.master("local[*]").appName("PySpark_Google_Colab").getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/Colab_Notebooks/IDT_Assignment"

file_path = "/content/drive/My Drive/Colab_Notebooks/IDT_Assignment/Patient_Satisfaction.csv"

Patient_Satisfaction.csv


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

#read file
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show()

+---------------+------------------+-----------------+------+---+
|     Department|       WaitingTime|SatisfactionScore|Gender|Age|
+---------------+------------------+-----------------+------+---+
|    Orthopedics| 46.64092191907339|                4| Other| 28|
|     Pediatrics| 55.88649935848038|                4|  Male| 42|
|General Surgery|  52.2151384760507|                4|Female| 71|
|     Pediatrics| 48.35826036418697|                4|  Male| 51|
|     Pediatrics|33.142883168320324|                4|  Male| 24|
|      Emergency|52.072025357039934|                4|  Male| 18|
|General Surgery|  73.2303674471255|                1|  Male| 28|
|General Surgery| 65.18130069232467|                1|  Male|  7|
|General Surgery| 68.89779939959095|                2|Female| 24|
|     Pediatrics| 37.33176485353222|                3| Other|  6|
|    Orthopedics| 30.15592769612129|                4|  Male|  0|
|General Surgery| 43.11319619850528|                3|  Male| 30|
|     Pedi

In [ ]:
#Schema
df.printSchema()
df.show(5)

root
 |-- Department: string (nullable = true)
 |-- WaitingTime: double (nullable = true)
 |-- SatisfactionScore: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)

+---------------+------------------+-----------------+------+---+
|     Department|       WaitingTime|SatisfactionScore|Gender|Age|
+---------------+------------------+-----------------+------+---+
|    Orthopedics| 46.64092191907339|                4| Other| 28|
|     Pediatrics| 55.88649935848038|                4|  Male| 42|
|General Surgery|  52.2151384760507|                4|Female| 71|
|     Pediatrics| 48.35826036418697|                4|  Male| 51|
|     Pediatrics|33.142883168320324|                4|  Male| 24|
+---------------+------------------+-----------------+------+---+
only showing top 5 rows



# Task 1: Calculate the Average Waiting Time by Department (using RDD)

In [ ]:
rdd = df.rdd

#Create key-value pairs where key is Department and value is a tuple (WaitingTime, 1)
#This will help us keep track of the total waiting time and the count of visits per department

#Sum up the total waiting time and the number of visits for each department
#so we can find the average waiting time for each department
waiting_time_rdd = rdd.map(lambda row: (row["Department"], (row["WaitingTime"], 1))) \
                      .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

#average waiting time: (sum_of_waiting_time / count)
avg_waiting_time_rdd = waiting_time_rdd.mapValues(lambda x: x[0] / x[1])

#show results
avg_waiting_time = avg_waiting_time_rdd.collect()
print("Average Waiting Time by Department (RDD):")
for dept, avg_time in avg_waiting_time:
    print(f"{dept}: {avg_time:.2f} minutes")

Average Waiting Time by Department (RDD):
Orthopedics: 46.64 minutes
Pediatrics: 45.73 minutes
General Surgery: 46.41 minutes
Emergency: 47.02 minutes
Cardiology: 44.60 minutes


# Task 2: Calculate the Average Waiting Time by Satisfaction Score (using RDD)


In [ ]:
#Task 2: Find the Average Waiting Time based on different Satisfaction Scores
#Create key-value pairs where key is SatisfactionScore and value is (WaitingTime, 1)
satisfaction_waiting_rdd = rdd.map(lambda row: (row["SatisfactionScore"], (row["WaitingTime"], 1)))

#Reduce by key to sum up waiting time and count for each satisfaction score
waiting_time_sum_rdd = satisfaction_waiting_rdd.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

#Calculate average waiting time: (sum_of_waiting_time / count)
avg_waiting_time_by_score_rdd = waiting_time_sum_rdd.mapValues(lambda x: x[0] / x[1])

#Collect and show results
avg_waiting_time_by_score = avg_waiting_time_by_score_rdd.collect()
print("Average Waiting Time by Satisfaction Score (RDD):")
for score, avg_time in avg_waiting_time_by_score:
    print(f"Satisfaction Score {score}: {avg_time:.2f} minutes")

Average Waiting Time by Satisfaction Score (RDD):
Satisfaction Score 4: 41.78 minutes
Satisfaction Score 1: 68.95 minutes
Satisfaction Score 2: 67.59 minutes
Satisfaction Score 3: 44.38 minutes
Satisfaction Score 5: 22.35 minutes


# Task 3: Calculate the Average Age and Gender Distribution by Department (using RDD)

In [ ]:
#Task 3: Calculate Average Age and Gender Distribution by Department
#Map to (Department, (Age, 1))
age_rdd = rdd.map(lambda row: (row["Department"], (row["Age"], 1)))

#Reduce by key to sum up ages and counts for each department
age_sum_count_rdd = age_rdd.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

#Calculate average age
avg_age_rdd = age_sum_count_rdd.mapValues(lambda x: x[0] / x[1])

#Collect and show results
avg_age = avg_age_rdd.collect()
print("Average Age by Department (RDD):")
for dept, avg_age in avg_age:
    print(f"{dept}: {avg_age:.2f} years")

#Task 3.2: Gender distribution by department
#Map to (Department, Gender), count occurrences
gender_rdd = rdd.map(lambda row: ((row["Department"], row["Gender"]), 1))

#Reduce by key to get gender counts
gender_count_rdd = gender_rdd.reduceByKey(lambda a, b: a + b)

#Collect and show results
gender_counts = gender_count_rdd.collect()
print("Gender Distribution by Department (RDD):")
for (dept, gender), count in gender_counts:
    print(f"Department: {dept}, Gender: {gender}, Count: {count}")

Average Age by Department (RDD):
Orthopedics: 42.39 years
Pediatrics: 43.52 years
General Surgery: 42.95 years
Emergency: 40.87 years
Cardiology: 40.72 years
Gender Distribution by Department (RDD):
Department: Orthopedics, Gender: Other, Count: 7000
Department: Pediatrics, Gender: Male, Count: 7300
Department: General Surgery, Gender: Female, Count: 6700
Department: Emergency, Gender: Male, Count: 5700
Department: General Surgery, Gender: Male, Count: 5600
Department: Pediatrics, Gender: Other, Count: 6400
Department: Orthopedics, Gender: Male, Count: 8000
Department: Pediatrics, Gender: Female, Count: 6700
Department: Emergency, Gender: Female, Count: 6700
Department: Cardiology, Gender: Male, Count: 5700
Department: Orthopedics, Gender: Female, Count: 5600
Department: Cardiology, Gender: Female, Count: 7600
Department: General Surgery, Gender: Other, Count: 6700
Department: Cardiology, Gender: Other, Count: 7700
Department: Emergency, Gender: Other, Count: 6600
